<a href="https://colab.research.google.com/github/chunribu/manim4stats/blob/main/2022/05/covariance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 协方差可视化 | 皮尔逊相关系数

**观看视频**

[![YouTube](https://icon-icons.com/downloadimage.php?id=168737&root=2699/PNG/32/&file=youtube_logo_icon_168737.png)](https://youtu.be/o_jAOtYDXrA) 
[![BiliBili](https://icon-icons.com/downloadimage.php?id=206792&root=3261/PNG/32/&file=bilibili_logo_icon_206792.png)](https://www.bilibili.com/video/BV1Bt4y1W7kB) 
[![iXigua](https://icon-icons.com/downloadimage.php?id=154582&root=2596/PNG/32/&file=xigua_icon_154582.png)](https://www.ixigua.com/7100162263713382919)


In [ ]:
# 安装Manim
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

In [ ]:
from manim import *

Manim Community v0.15.2

In [ ]:
colors = [BLUE, TEAL,  GREEN, YELLOW, GOLD, RED, MAROON]

def get_dots():
    dots = [Dot(color=i) for i in colors]
    labels = [SingleStringMathTex(f'x_{i+1}', color=colors[i], font_size=20) for i in range(7)]
    dots_g = VGroup(*dots, *labels).arrange_in_grid(rows=2)
    box = SurroundingRectangle(dots_g, color=WHITE, buff=0.3)
    dots_g_b = VGroup(box, dots_g)
    return dots, dots_g_b

def get_nl():
    return NumberLine(
        x_range=[1,7,1],
        length=5*6/7,
        include_numbers=True
    )

def get_rectangle_corners(bottom_left, top_right): #(-1,0),(1,2)
    return [
        (top_right[0], top_right[1]),    #(1,2) TR
        (bottom_left[0], top_right[1]),   #(-1,2) TL
        (bottom_left[0], bottom_left[1]),  #(-1,0) BL
        (top_right[0], bottom_left[1]),   #(1,0) BR
    ]


In [ ]:
%%manim -ql -v WARNING View0

class View0(Scene):
    def construct(self):
        self.wait()

        dots, dots_g_b = get_dots()
        self.play(Create(dots_g_b, run_time=2))
        self.wait()
        self.play(dots_g_b.animate.to_edge(UL, buff=MED_LARGE_BUFF))

        nl = get_nl()
        _dots = [d.copy() for d in dots]
        self.play(*[d.animate.move_to(nl.n2p(i+1)) for i,d in enumerate(_dots)])

        self.play(Write(nl))

        nl_d = VGroup(nl, *_dots)
        self.play(nl_d.animate.shift(DOWN*2.5+RIGHT*0.35))
        self.wait()

        nl_d_y = nl_d.copy().move_to(ORIGIN+RIGHT*0.35)
        self.play(Write(nl_d_y))
        self.wait()

        self.play(nl_d_y.animate.move_to(nl_d.get_center()))
        self.play(Rotate(nl_d_y, angle=PI/2, about_point=nl.n2p(0)))

        self.play(FadeOut(nl), FadeOut(nl_d_y.submobjects[0]))

        ax = Axes(
            x_range=[0, 7],
            y_range=[0, 7],
            x_length=5,
            y_length=5,
            axis_config={
                "include_numbers": True,
                "include_tip": False
            },
        )
        labels = ax.get_axis_labels(x_label="x", y_label="y")
        self.play(Create(ax))
        self.play(Write(labels))
        self.wait()

        _dots_y = nl_d_y.submobjects[1:]
        traces = [ax.get_lines_to_point(ax.c2p(i+1,i+1),color=GRAY) for i in range(7)]
        self.play(*[d.animate(run_time=2).move_to(ax.c2p(i+1,i+1)) for i,d in enumerate(_dots)],
                  *[d.animate(run_time=2).move_to(ax.c2p(i+1,i+1)) for i,d in enumerate(_dots_y)],
                  *[Write(l,run_time=3.5) for l in traces])
        self.remove(*_dots_y)

        mu_x = ax.get_vertical_line(ax.c2p(4,7), line_func=Line, color=BLUE_D)
        mu_x_label = SingleStringMathTex(r'\mu_x', color=BLUE_D).next_to(mu_x, UP)
        self.play(Write(mu_x))
        self.play(Write(mu_x_label, run_time=0.5))

        mu_y = ax.get_horizontal_line(ax.c2p(7,4), line_func=Line, color=YELLOW_D)
        mu_y_label = SingleStringMathTex(r'\mu_y', color=YELLOW_D).next_to(mu_y, RIGHT)
        self.play(Write(mu_y))
        self.play(Write(mu_y_label, run_time=0.5))
        self.wait(2)

        delta_xs = [Arrow(start=ax.c2p(i+1,i+1), end=ax.c2p(4,i+1), color=BLUE_D, buff=0) for i in range(7)]
        delta_ys = [Arrow(start=ax.c2p(i+1,i+1), end=ax.c2p(i+1,4), color=YELLOW_D, buff=0) for i in range(7)]
        rects = []
        for i in range(7):
            if i < 3:
                rects.append(Polygon(*[ax.c2p(*c) for c in get_rectangle_corners((i+1,i+1), (4,4))], fill_color=colors[i], fill_opacity=0.5))
                rects[-1].stroke_width = 0
            elif i > 3:
                rects.append(Polygon(*[ax.c2p(*c) for c in get_rectangle_corners((4,4), (i+1,i+1))], fill_color=colors[i], fill_opacity=0.5))
                rects[-1].stroke_width = 0
            else:
                rects.append(Square(side_length=0.2, color=colors[i]).move_to(ax.c2p(4,4)).rotate(PI/4))

        self.play(*[Write(i) for i in delta_xs])
        self.wait()
        self.play(*[Write(i) for i in delta_ys])
        self.wait()
        for i in rects:
            self.play(Create(i, run_time=0.5))

        ax_g = VGroup(ax, *labels, *_dots, mu_x, mu_x_label, mu_y, mu_y_label, *traces, *delta_xs, *delta_ys, *rects)
        self.play(ax_g.animate.to_edge(DL, buff=SMALL_BUFF))

        t_cal = MathTex('(',r'3\times3','+',r'2\times2','+',r'1\times1','+',r'0\times0','+',r'1\times1','+',r'2\times2','+',r'3\times3',')',r'\times\frac{1}{7}=4',
                        font_size=34).to_edge(UR)
        t_cov = MathTex(r'Cov(x,y)=\frac{\sum_{i=1}^N (x_{i}-\mu_x)(y_{i}-\mu_y)}{N}').to_edge(UR).shift(DOWN*2)
        t_var = MathTex(r'\sigma^2=\frac{\sum_{i=1}^N (x_{i}-\mu)^2}{N}').next_to(t_cov, DOWN)
        self.play(*[rects[i].animate.move_to(t_cal.submobjects[j].get_center()) for i, j in enumerate([1,3,5,7,9,11,13])])
        self.play(*[FadeOut(rects[i], run_time=0.5) for i in range(7)],
                  *[Create(t_cal.submobjects[j]) for j in [1,3,5,7,9,11,13]])
        self.wait()
        self.play(*[Create(t_cal.submobjects[j]) for j in [2,4,6,8,10,12]])
        self.play(*[Create(t_cal.submobjects[j]) for j in [0,-2]])
        self.play(Create(t_cal.submobjects[-1]))

        c_cov = Text('协方差(Covariance)').scale(0.5).to_edge(DOWN, buff=MED_LARGE_BUFF).shift(RIGHT)
        self.play(Write(c_cov))

        self.play(Write(t_cov))
        self.wait()
        self.play(Unwrite(c_cov))

        img = ImageMobject('../stat-variance/cover.jpg')
        img.height = 4
        self.play(FadeIn(img, run_time=0.5))
        self.play(Circumscribe(img, color=WHITE))
        self.wait()
        self.play(FadeOut(img, run_time=0.5), Write(t_var, run_time=1.5))
        self.wait(2)

        quadrants = dict()
        quadrants[1] = Polygon(*[ax.c2p(*c) for c in get_rectangle_corners((4,4), (8,8))], fill_color=RED, fill_opacity=0.5)
        quadrants[2] = Polygon(*[ax.c2p(*c) for c in get_rectangle_corners((0,4), (4,8))], fill_color=BLUE, fill_opacity=0.5)
        quadrants[3] = Polygon(*[ax.c2p(*c) for c in get_rectangle_corners((0,0), (4,4))], fill_color=RED, fill_opacity=0.5)
        quadrants[4] = Polygon(*[ax.c2p(*c) for c in get_rectangle_corners((4,0), (8,4))], fill_color=BLUE, fill_opacity=0.5)

        self.play(Unwrite(ax), Unwrite(labels))
        self.play(FadeIn(quadrants[1]), FadeIn(quadrants[3]))
        self.wait(2)
        self.play(FadeIn(quadrants[2]), FadeIn(quadrants[4]))
        self.wait(2)

        self.play(*[FadeOut(quadrants[i]) for i in [1,2,3,4]])
        ax_g = ax_g[3:-len(rects)]
        self.play(ax_g.animate.scale(0.5).next_to(dots_g_b, DOWN))
        self.play(t_cov.animate.scale(0.8).to_edge(LEFT).shift(DOWN*2.5))
        self.play(t_var.animate.scale(0.8).next_to(t_cov, DOWN), run_time=0.5)
        self.wait()

        ul = Underline(t_cal)
        self.play(Write(ul))
        t_sigma_xy = MathTex(r'\sigma_x\sigma_y', font_size=34).next_to(ul, DOWN)
        self.play(Write(t_sigma_xy))
        self.wait()

        t_sigma = MathTex(r'\sigma=\sqrt{\sigma^2}')
        self.play(Write(t_sigma))
        self.wait()

        t_sigma_2 = MathTex(r'\sigma^2').move_to(t_var.get_left())
        self.play(t_sigma_2.animate.move_to(t_sigma_xy.get_center()))
        self.play(FadeOut(t_sigma_2, run_time=0.3))

        t_2x2 = MathTex(r'2\times2', font_size=34).move_to(t_sigma_xy.get_center())
        self.play(ReplacementTransform(t_sigma_xy, t_2x2), FadeOut(t_sigma))

        c_pearson = Text('Pearson correlation coefficient').scale(0.5).to_edge(DOWN, buff=MED_LARGE_BUFF)
        self.play(Write(c_pearson))

        t_pearson = MathTex(r'\rho_{xy}=\frac{Cov(x,y)}{\sigma_x\sigma_y}')
        self.play(Write(t_pearson))
        self.play(Unwrite(c_pearson))

        t_pearson_ = MathTex(r'-1\leq\rho\leq1').next_to(t_pearson, buff=0.7)
        self.play(Write(t_pearson_))

        self.play(t_pearson.animate.scale(1.2).shift(RIGHT*1.5), t_pearson_.animate.scale(0.8).shift(RIGHT*1.5))

        self.wait(4)